In [1]:
import pandas as pd
import numpy as np
import pulp
import os
os.getcwd()
os.chdir("..")

# Process data

In [2]:
raw_workmode = pd.read_excel('InputData.xlsx', sheet_name="Mode")
raw_demand = pd.read_excel('InputData.xlsx', sheet_name="DemandV2") # ver2
workmode = {(row['Mode'].item(), row['Shift'].item()): row['Capa'].item() for _, row in raw_workmode.iterrows()}
demand = {row['Week'].item(): row['Demand'].item() for _, row in raw_demand.iterrows()} # ver2

In [3]:
WeekIndex = raw_demand.Week.tolist()
ShiftIndex = [1,2,3]
bigM = 99999

In [4]:
raw_workmode

,Mode,Shift,Capa,Hours
0,1,1,7182,8
1,1,2,9453,10
2,1,3,11261,12
3,2,1,11271,8
4,2,2,13923,10
5,2,3,16576,12


In [5]:
numS = 22
numC = 12
FC = {
    (i, j): raw_workmode.loc[(raw_workmode['Mode'] == i) & (raw_workmode['Shift'] == j), 'Hours'].item() * (
        8.3 * 1.5 * 12 if (j in [2, 3] and i == 1) else
        8.3 * 1.5 * 17 if (j in [2, 3] and i == 2) else
        8.3 * 12 if (j in [1] and i == 1) else
        8.3 * 17 if (j in [1] and i == 2) else
        0)
    for i in [1, 2]
    for j in ShiftIndex
    if not raw_workmode.loc[(raw_workmode['Mode'] == i) & (raw_workmode['Shift'] == j), 'Hours'].empty}

# Algorithm

In [19]:
# Create the model
model = pulp.LpProblem("3PL_Workforce_Allocation", pulp.LpMinimize)

# Define binary decision variables x (3D) and y (2D)
m_conso = pulp.LpVariable.dicts("m_conso", (ShiftIndex, WeekIndex), lowBound=0, cat='Integer')
m_sepa = pulp.LpVariable.dicts("m_sepa", (ShiftIndex, WeekIndex), lowBound=0, cat='Integer')
# total_shifts = pulp.LpVariable.dicts("total_shifts", WeekIndex, cat='Integer')
x = pulp.LpVariable.dicts("x", WeekIndex, cat='Binary')
# delta_conso = pulp.LpVariable.dicts("delta_conso", (ShiftIndex, WeekIndex), lowBound=0, upBound=1, cat='Integer')
# delta_sepa = pulp.LpVariable.dicts("delta_sepa", (ShiftIndex, WeekIndex), lowBound=0, upBound=1, cat='Integer')

# Objective function: 
model += pulp.lpSum(numC*FC[1, sh] * m_conso[sh][w] for sh in ShiftIndex for w in WeekIndex) + \
         pulp.lpSum(numS*FC[2, sh] * m_sepa[sh][w] for sh in ShiftIndex for w in WeekIndex)

# Constraint

# Can't work more than 3 shifts 3 per week
for w in WeekIndex:
    model += m_conso[3][w] + m_sepa[3][w] <= 3

# Satisfy demand ver 2
for w in WeekIndex:
    model += pulp.lpSum(workmode[1, sh]*m_conso[sh][w] for sh in ShiftIndex) + pulp.lpSum(workmode[2, sh]*m_sepa[sh][w] for sh in ShiftIndex) >= demand[w]

# Min 5 days a week
for w in WeekIndex:
    model += pulp.lpSum(m_conso[sh][w] for sh in ShiftIndex) + pulp.lpSum(m_sepa[sh][w] for sh in ShiftIndex) >= 5

# Max 7 days a week
for w in WeekIndex:
    model += pulp.lpSum(m_conso[sh][w] for sh in ShiftIndex) + pulp.lpSum(m_sepa[sh][w] for sh in ShiftIndex) <= 7

# If sum = 7, then apply 2 times wages
## If the total of m_conso and m_sepa is 7, activate x[w]
for w in WeekIndex:
    total_shifts = pulp.lpSum(m_conso[sh][w] + m_sepa[sh][w] for sh in ShiftIndex)
    if total_shifts == 7:
        x[w] = 1
    else:
        x[w] = 0
    # model += total_shifts - 7 <= bigM * (1 - x[w])
    # model += total_shifts - 7 >= -bigM * (1 - x[w])

# ## If x[w] is 1, then either m_conso or m_sepa must increase by exactly 1
# for w in WeekIndex:
#     model += pulp.lpSum(delta_conso[sh][w] for sh in ShiftIndex) + pulp.lpSum(delta_sepa[sh][w] for sh in ShiftIndex) == x[w]

# ## Only add to available mode
# for w in WeekIndex:
#     for sh in ShiftIndex:
#         model += m_conso[sh][w] >= delta_conso[sh][w] 
# for w in WeekIndex:
#     for sh in ShiftIndex:
#         model += m_sepa[sh][w] >= delta_sepa[sh][w]

# # If z[w] is 0, then delta_conso and delta_sepa must both be 0 for all shifts
# for w in WeekIndex:
#     for sh in ShiftIndex:
#         model += delta_conso[sh][w] <= x[w]
#         model += delta_sepa[sh][w] <= x[w]

# Solve the problem using COIN_CMD without the tol argument
solver = pulp.PULP_CBC_CMD(gapRel=0.00001)  # Built-in solver
model.solve(solver)

# Collect the results into a list of dictionaries for x_ijk (3D)
results_conso = []
for sh in ShiftIndex:
    for w in WeekIndex:
        results_conso.append({
            'shift': sh,
            'week': w,
            'choose': m_conso[sh][w].varValue
        })

# res_delta_conso = []
# for sh in ShiftIndex:
#     for w in WeekIndex:
#         res_delta_conso.append({
#                 'shift': sh,
#                 'week': w,
#                 'choose': delta_conso[sh][w].varValue
#             })

# Collect the results into a list of dictionaries for y_ij (2D)
results_sepa = []
for sh in ShiftIndex:
    for w in WeekIndex:
        results_sepa.append({
            'shift': sh,
            'week': w,
            'choose': m_sepa[sh][w].varValue
        })

# res_delta_sepa = []
# for sh in ShiftIndex:
#     for w in WeekIndex:
#         res_delta_sepa.append({
#                 'shift': sh,
#                 'week': w,
#                 'choose': delta_sepa[sh][w].varValue
#             })

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/lamjackie/miniconda3/envs/ych-wf-allocation/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/4v/qgjl0ptd02j55mwtql93llqr0000gn/T/350ec4a9ee55470ea39ff23954387a47-pulp.mps -ratio 1e-05 -timeMode elapsed -branch -printingOptions all -solution /var/folders/4v/qgjl0ptd02j55mwtql93llqr0000gn/T/350ec4a9ee55470ea39ff23954387a47-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 37 COLUMNS
At line 342 RHS
At line 375 BOUNDS
At line 424 ENDATA
Problem MODEL has 32 rows, 48 columns and 160 elements
Coin0008I MODEL read with 0 errors
ratioGap was changed from 0 to 1e-05
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1.03471e+06 - 0.00 seconds
Cgl0004I processed model has 24 rows, 48 columns (48 integer (0 of which binary)) and 112 elements
Cutoff increment increased from 1e-05 to 16.5999
Cbc0012I Integer solution of 

In [20]:
# Calculate and print total shifts for each week
for w in WeekIndex:
    total_shifts_value = sum(m_conso[sh][w].varValue + m_sepa[sh][w].varValue for sh in ShiftIndex)
    print(f"Total shifts for week {w}: {total_shifts_value}")



Total shifts for week 1: 7.0
Total shifts for week 2: 7.0
Total shifts for week 3: 7.0
Total shifts for week 4: 7.0
Total shifts for week 5: 7.0
Total shifts for week 6: 7.0
Total shifts for week 7: 7.0
Total shifts for week 8: 5.0


In [21]:
for w in WeekIndex:
    print(x[w])

1
1
1
1
1
1
1
1


In [ ]:
df_results_conso=pd.DataFrame(results_conso)
df_results_conso['mode'] = "c"
df_results_sepa = pd.DataFrame(results_sepa)
df_results_sepa['mode'] = "s"
result = pd.concat([df_results_sepa, df_results_conso])
result_pivot=pd.DataFrame(result.pivot_table(index=["mode", "shift"], columns=["week"], values=["choose"], aggfunc="sum").reset_index(drop=False))
result_pivot

In [ ]:
workmode

In [ ]:
raw_demand

In [9]:
with pd.ExcelWriter("result_v2.xlsx") as writer:
   
    # use to_excel function and specify the sheet_name and index 
    # to store the dataframe in specified sheet
    result_pivot.to_excel(writer, sheet_name="Pivot")
    result.to_excel(writer, sheet_name="Results", index=False)
    raw_workmode.to_excel(writer, sheet_name="Workforce", index=False)
    raw_demand.to_excel(writer, sheet_name="Demand", index=False)

In [ ]:
df_results_sepa